<h1> FlyWave Classification Notebook </h1>

A quick tutorial on how to setup and run a deep learning classifier using Keras for Hand Gesture classification.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import io
import os
from PIL import Image

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [1]:
DATAPATH = 'images/'
HEIGHT = 360
WIDTH = 640
# input image dimensions
img_rows, img_cols = HEIGHT, WIDTH

Place preprocessing methods into this method. We will use this to return a modified version of the image so that we can add them to our dataset.

In [3]:
def preprocess(img):
    img = cv2.resize(img, (img.shape[0]//2, img.shape[1]//2))
    return img

Load data from the "images" directory by reading in the images. We can resize the images to half their width and height since we don't need all of the color data (nor do we want our model to take forever to train).

In [ ]:
data = []
labels = []
label_index = 0 

for label in os.listdir(DATAPATH):
    if label == 'ok' or label == 'rock':
        img_dir = DATAPATH + label + '/'
        for fn in os.listdir(img_dir):
            img = cv2.imread(img_dir + fn, 1)
            img = preprocess(img)
            data.append(img)
            labels.append(label_index)
        label_index += 1    

print("Finished loading data")

Convert the data into numpy arrays to input into our model, and save them.

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
np.save('data.npy', data)
np.save('labels.npy', labels)

In [5]:
#Use these to load your data
data = np.load('data.npy')
labels = np.load('labels.npy')

Partition the data into our training set and test set. We usually use 80% of our data to train our model on and 20% of our data to test it on, so we don't over fit our model. SKLearn has a handy function for doing this.

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=6)

We then need to set up our data to put into our Keras model. We set the parameters in this cell (batch size, epochs, etc) and also normalize the image data so that we have data from [0,255] to [0,1].

In [8]:
batch_size = 128
num_classes = 2
epochs = 12
channels = 3


# the data, split between train and test sets

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (400, 720, 1280, 3)
400 train samples
100 test samples


<h3>Model</h3>


We can now create our neural network model. In Keras, we initialize a sequence of neural network layers with Sequential(). Once we have our base model, we can then simply add our desired layers. Keras provides a handful of methods that we can add to our model, which include:

Conv2D(# hidden units, kernel_size, activation, input_shape)
MaxPooling2D(pool_size)
Dropout
Flatten
Dense(# hidden units, activation)

Any of these layers can be added in any combination with model.add(LAYER). 

<b>Activation Functions</b> <br>
softmax - map hidden units to probabilities [0,1]
relu - useful for images in convolutional layers



In [ ]:
model = Sequential()
#Conv2D (32 hidden layers, kernel=[3,3]) -> relu -> #Conv2D (32 hidden layers, kernel=[3,3]) -> relu ->
#maxpooling (2,2) -> dropout -> flatten (change from 3D to 1D) -> fully connected layer (128 hidden layers) -> relu -> 
#dropout -> Classification using softmax


<h3>Validation and Model Evaluation</h3>

Once the model is complete, we can compile it by specifying the necessary parameters. Loss specifies what loss function the model is trying to minimize. The optimizer is the algorithm that is used to lower the loss. 


You can now train the model by calling the "fit" method. Now watch your model train and see the results!

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 400 samples, validate on 100 samples
Epoch 1/12


In [ ]:
with open("results.txt", "w") as f:
    f.write(score)

In [ ]:
model.save("model_0.h5")